In [1]:
import pandas as pd

import matplotlib.pyplot as plt

import numpy as np

from citipy import citipy

In [2]:
# generating the random latitude and longitude arrays
# storing them in variables
# packing the latitude and longitude values as pairs using the zip function

lats = np.random.uniform(-90.000, 90.000, size=1500)

lngs = np.random.uniform(-180.000, 180.000, size=1500)

lats_lngs = zip(lats, lngs)

lats_lngs

# The output of this cell will be a zip object in memory

# Notes on above code block

- The zip function packs each pair of lats and lngs with the same index into a tuple
    - Here, that produces 1500 tuples of paired latitudes and longitues 
        - each latitude and longitude in a tuple can be accessed by the index of 0 and 1 respectively
    - cf the jupyter notebook API_Practice.ipynb for practice packing and unpacking zipped tuples

In [3]:
# adding the latitude and longitudes to a list
# means we only need to create the random sets once
# NOTE you can only unzup a zipped tuple once before it's removed from computer's memory.

coordinates = list(lats_lngs)

In [4]:
# creating an empty list to hold the cities

cities = []

# identifying nearest city for each latitude and longitude combination

for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # if the city name is unique, add to the cities []
    # The for-loop could find duplicate cities, and we only want unique cities
    
    if city not in cities:
        cities.append(city)
        
# print the city count to confirm sufficient count:
# Note that citipy finds the nearest city to the latitude and longitude pair with a population of 500+

print(len(cities))

print(cities[3])

602
hobart


# Getting Started with OpenWeather API

## JavaScript Object Notation Format for API Data

- Retrieving data from a website requires a requset which returns data in a text format (not tsv or csv)
- JavaScript Object Notation (JSON)
    - data within this sort of object opens and closes with curly braces {}, like a Python Dictionary
    - inside the JSON object, is a collection of dictionaries and arrays.

- To request JSON data over the internet, need the Requests Library in Python 
    - Anaconda comes witih version 2.22 or later installed.
    
[Doc for Requests Library](https://requests.kennethreitz.org/en/master/)]

[Quickstart Doc](https://requests.kennethreitz.org/en/master/user/quickstart/#make-a-request)

# Get the City Weather Data

## Pseudocoding 

1. Import the dependencies and initialize counters and an empty list that will hold weather data
2. Loop through the cities list
3. Group the cities in sets of 50 to log the process as we find weather data for each city
    - two counters will be need: one to log the city count from 1-50 and the other for the sets
4. Build the city_url or endpoint for each city
5. log the URL and the record and set numbers
6. Make an API request for each city
7. Parse the JSON weather data for the following: 
    - City, country, and date
    - Lat and long
    - Max temp
    - Humidity
    - Cloudiness 
    - Wind speed
8. Add the data to a list in a deictionary format and then convert to a DataFrame

In [5]:
# here, declare empty list city_data = []
# add print statement that ref the beginning of the logging 
# create counters for the record numbers 1-50
# set the counter

import requests

from config import weather_api_key

import time
from datetime import datetime

url = "https://api.openweathermap.org/data/2.5/weather?&appid=" + weather_api_key

#city_weather = url + "&q=" + "ushuaia"

#print(city_weather)

In [6]:
# creating the empty list to hold weather data

city_data = []

# Print the beginning of the logging

print("Beginning Data Retrieval    ")
print("----------------------------")

# create the counters

record_count = 1

set_count = 1

# here setting the counters at 1 because we want the first iteration of logging for each recorded response to st @ 1

# Now iterating through list of cities and building the URL for reach city, grouping in sets of 50
# use for i in range(len(cities)) and index to notify when reach 50
# at 50, pause for 60 sec using time.sleep(60) free OpenWeatherMap API allows for 60calls/min
# the pause prevents time-out errors
# retrieve city from cities list and add to the city_url using indexing:

# here using the enumerate method because it will return the city name - vital for ural
# the in range() would have returned an index. 
# enumerate doc: https://docs.python.org/3/library/functions.html#enumerate

for i, city in enumerate(cities):
# you have the 2 variables here because of the way enumerate ret. tuple w/ count and value    
    # Group cities in sets of 50 for logging
    if (i % 50 == 0 and i >= 50):
        
        set_count += 1
        
        record_count = 1
        
        time.sleep(60)
        
    # Create endpoint URL with each city
    
    
    city_url = url + "&q=" + city.replace(" ", "+")
    
    # Log the URL, record and set numbers and the city
    
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add one to the record_count
    
    record_count += 1
    
    # Run API request for each of the cities using a try except
    
    try: 
        # Parse the JSON and retrieve the data
        city_weather = requests.get(city_url).json()
        
        # Parse the needed data
        
        city_lat = city_weather['coord']['lat']
        
        city_lng = city_weather['coord']['lon']
        
        city_max_temp = city_weather['main']['temp_max']
        
        city_humidity = city_weather['main']['humidity']
        
        city_clouds = city_weather['clouds']['all']
        
        city_wind = city_weather['wind']['speed']
        
        city_country = city_weather['sys']['country']
        
        # Convert the date to ISO standard
        
        city_date = datetime.utcfromtimestamp(city_weather['dt']).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append city information to the city_data list
        
        city_data.append({"City": city.title(),
                         "Lat": city_lat, 
                         "Lng": city_lng,
                         "Max Temp": city_max_temp,
                         "Humidity": city_humidity,
                         "Cloudiness": city_clouds, 
                         "Wind Speed": city_wind,
                         "Country": city_country,
                         "Date": city_date})
        
    # if error is experienced, skip the city:
    
    except:
        
        print("City not found. Skipping...")
        
        # pass is a general purpose statement to handle all errors encountered and let prog. continue
        # in reality, pass statement is not best practice
        # better practice is to handle each error as it comes (add another try block or print error)
        pass
    
# Indicate that Data Loading is complete:

print('-------------------------------')

print('Data Retrieval Complete')

print('-------------------------------')
    
    

Beginning Data Retrieval    
----------------------------
Processing Record 1 of Set 1 | punta arenas
Processing Record 2 of Set 1 | hofn
Processing Record 3 of Set 1 | jamestown
Processing Record 4 of Set 1 | hobart
Processing Record 5 of Set 1 | faanui
Processing Record 6 of Set 1 | zhangye
Processing Record 7 of Set 1 | port elizabeth
Processing Record 8 of Set 1 | victoria
Processing Record 9 of Set 1 | khatanga
Processing Record 10 of Set 1 | mar del plata
Processing Record 11 of Set 1 | salalah
Processing Record 12 of Set 1 | vardo
Processing Record 13 of Set 1 | bethel
Processing Record 14 of Set 1 | lorengau
Processing Record 15 of Set 1 | beringovskiy
Processing Record 16 of Set 1 | bengkulu
Processing Record 17 of Set 1 | meihekou
Processing Record 18 of Set 1 | severnyy
Processing Record 19 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 20 of Set 1 | malabor
Processing Record 21 of Set 1 | tuatapere
Processing Record 22 of Set 1 | rikitea
Processin

Processing Record 37 of Set 4 | saskylakh
Processing Record 38 of Set 4 | sovetskiy
Processing Record 39 of Set 4 | kamenskoye
City not found. Skipping...
Processing Record 40 of Set 4 | cabo san lucas
Processing Record 41 of Set 4 | murgab
Processing Record 42 of Set 4 | grozesti
Processing Record 43 of Set 4 | novikovo
Processing Record 44 of Set 4 | nouadhibou
Processing Record 45 of Set 4 | mahanoro
Processing Record 46 of Set 4 | santa fe
Processing Record 47 of Set 4 | mitsamiouli
Processing Record 48 of Set 4 | lago da pedra
Processing Record 49 of Set 4 | ancud
Processing Record 50 of Set 4 | antofagasta
Processing Record 1 of Set 5 | nikolskoye
Processing Record 2 of Set 5 | tilichiki
Processing Record 3 of Set 5 | shahpura
Processing Record 4 of Set 5 | aksu
Processing Record 5 of Set 5 | inhambane
Processing Record 6 of Set 5 | vestmanna
Processing Record 7 of Set 5 | makungu
Processing Record 8 of Set 5 | dikson
Processing Record 9 of Set 5 | tecpan
Processing Record 10 of 

Processing Record 26 of Set 8 | merrill
Processing Record 27 of Set 8 | ribeira grande
Processing Record 28 of Set 8 | san patricio
Processing Record 29 of Set 8 | kavaratti
Processing Record 30 of Set 8 | banyo
Processing Record 31 of Set 8 | zaria
Processing Record 32 of Set 8 | bargarh
Processing Record 33 of Set 8 | seoul
Processing Record 34 of Set 8 | vaitape
Processing Record 35 of Set 8 | poya
Processing Record 36 of Set 8 | mezen
Processing Record 37 of Set 8 | tondano
Processing Record 38 of Set 8 | sabha
Processing Record 39 of Set 8 | maniitsoq
Processing Record 40 of Set 8 | kintampo
Processing Record 41 of Set 8 | panlaitan
Processing Record 42 of Set 8 | kupang
Processing Record 43 of Set 8 | chubbuck
Processing Record 44 of Set 8 | port-gentil
Processing Record 45 of Set 8 | tutoia
Processing Record 46 of Set 8 | coolum beach
Processing Record 47 of Set 8 | huaraz
Processing Record 48 of Set 8 | rize
Processing Record 49 of Set 8 | padang
Processing Record 50 of Set 8 |

Processing Record 16 of Set 12 | kilindoni
Processing Record 17 of Set 12 | derzhavinsk
Processing Record 18 of Set 12 | stephenville crossing
Processing Record 19 of Set 12 | fare
Processing Record 20 of Set 12 | ndioum
Processing Record 21 of Set 12 | hun
Processing Record 22 of Set 12 | asau
Processing Record 23 of Set 12 | bogatoye
Processing Record 24 of Set 12 | hanstholm
Processing Record 25 of Set 12 | okha
Processing Record 26 of Set 12 | rocha
Processing Record 27 of Set 12 | cerro de pasco
Processing Record 28 of Set 12 | katiola
Processing Record 29 of Set 12 | lokosovo
Processing Record 30 of Set 12 | saint-francois
Processing Record 31 of Set 12 | saint-joseph
Processing Record 32 of Set 12 | rangapara
Processing Record 33 of Set 12 | karratha
Processing Record 34 of Set 12 | tomatlan
Processing Record 35 of Set 12 | roros
Processing Record 36 of Set 12 | sivas
Processing Record 37 of Set 12 | manadhoo
Processing Record 38 of Set 12 | gambela
Processing Record 39 of Set 1

In [7]:
#print(city_url)

In [8]:
# city_weather = requests.get(city_url).json()
# print(city_weather)

# Try and Except block

- help you try a block of code, if it throws an error, it goes to the except block
- The try block is the code you would want to run. 
    - set up is like a conditional statement - indention 
    - you can tell the except block the exact error 
    - good idea to write a print statement to communicate you've received an error
    - then add a continue so that it goes back to the loop.
    
[Doc for try-except](https://docs.python.org/3.7/tutorial/errors.html)

In [9]:
# converting the city_data list created from API request above to pandas DF

city_data_df = pd.DataFrame(city_data)

city_data_df.head(15)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Punta Arenas,-53.1500,-70.9167,275.21,80,75,4.63,CL,2022-07-14 11:47:21
1,Hofn,64.2539,-15.2082,285.16,76,20,4.63,IS,2022-07-14 11:50:15
2,Jamestown,42.0970,-79.2353,291.49,83,0,2.06,US,2022-07-14 11:49:32
3,Hobart,-42.8794,147.3294,282.07,93,75,0.00,AU,2022-07-14 11:50:06
4,Faanui,-16.4833,-151.7500,298.75,72,11,10.37,PF,2022-07-14 11:47:30
5,Zhangye,38.9342,100.4517,295.80,51,100,3.54,CN,2022-07-14 11:50:16
6,Port Elizabeth,-33.9180,25.5701,289.87,74,75,8.75,ZA,2022-07-14 11:50:16
7,Victoria,22.2855,114.1577,305.47,75,95,3.58,HK,2022-07-14 11:49:04
8,Khatanga,71.9667,102.5000,281.86,74,100,3.21,RU,2022-07-14 11:50:17
9,Mar Del Plata,-38.0023,-57.5575,283.20,90,0,7.15,AR,2022-07-14 11:48:28
